### Author: Hoang Chu

### Imports

In [7]:
%pip install -r dependencies.txt

  Using cached aiofiles-22.1.0-py3-none-any.whl (14 kB)
  Using cached aiosqlite-0.18.0-py3-none-any.whl (15 kB)
  Using cached anyio-3.6.2-py3-none-any.whl (80 kB)
  Using cached argon2_cffi-21.3.0-py3-none-any.whl (14 kB)
  Using cached argon2_cffi_bindings-21.2.0-cp38-abi3-macosx_10_9_universal2.whl (53 kB)
  Using cached arrow-1.2.3-py3-none-any.whl (66 kB)
  Using cached asttokens-2.2.1-py2.py3-none-any.whl (26 kB)
  Using cached attrs-22.2.0-py3-none-any.whl (60 kB)
  Using cached Babel-2.12.1-py3-none-any.whl (10.1 MB)
  Using cached backcall-0.2.0-py2.py3-none-any.whl (11 kB)
  Using cached bardapi-0.1.38-py3-none-any.whl.metadata (21 kB)
  Using cached beautifulsoup4-4.11.2-py3-none-any.whl (129 kB)
  Using cached bleach-6.0.0-py3-none-any.whl (162 kB)
  Using cached browser_cookie3-0.19.1-py3-none-any.whl.metadata (632 bytes)
  Using cached cachetools-5.3.2-py3-none-any.whl.metadata (5.2 kB)
  Using cached certifi-2022.12.7-py3-none-any.whl (155 kB)
  Using cached cffi-1.15.1

In [8]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import requests
import scipy
import importlib
import helper, consts
importlib.reload(consts)
importlib.reload(helper)
import glob

### CONSTANTS

In [9]:
# User Specific Constants:
USER = "HENRY"
FILE_PATH = consts.PATH_MAP[USER]

ROW = consts.ROW
COL = consts.COL
CLEANED_DATA_PATH = consts.CLEANED_DATA_PATH
RESPONSE_NAME = consts.RESPONSE_NAME


### Quick Functions

### Read data (8 predictors)

In [10]:
# Main Reading Script
train_start, train_end = helper.get_train_from_testday('20150701')
x_cols = ["rrirpnxm_nt_0", "rrirpnxm_lst15_0","rrirpnxm_lsthrx15_0", "rrirpnxm_toxhr_0", 
          "relvol_nt_0", "relvol_lst15_0","relvol_lsthrx15_0", "relvol_toxhr_0"
          ]

train_df = helper.get_df(train_start, train_end, x_cols, FILE_PATH)
print(train_df.head())
# train_df.to_csv(CLEANED_DATA_PATH, index_label=False)


### Read data (all predictors)

In [ ]:
# train_start, train_end = helper.get_train_from_testday('20150701')
# x_cols = "cftorrrelstd_open_0,liqlog_open_0,llirpnxm_am_1,llirpnxm_dy_12to16,llirpnxm_dy_17to21,llirpnxm_dy_1to3,llirpnxm_dy_4to6,llirpnxm_dy_7to11,llirpnxm_lst15_0,llirpnxm_lsthrx15_0,llirpnxm_md_1,llirpnxm_nt_0,llirpnxm_nt_1,llirpnxm_nt_12to16,llirpnxm_nt_17to21,llirpnxm_nt_1to3,llirpnxm_nt_4to6,llirpnxm_nt_7to11,llirpnxm_pm_1,llirpnxm_toxhr_0,mocrelvol_open_1,mocrelvol_open_13to17,mocrelvol_open_18to22,mocrelvol_open_2to4,mocrelvol_open_5to7,mocrelvol_open_8to12,momr10d_open_0,momr50dx10d_open_0,moorelvol_open_0,moorelvol_open_12to16,moorelvol_open_17to21,moorelvol_open_1to3,moorelvol_open_4to6,moorelvol_open_7to11,nnetticksrelmultstdev_am_1,nnetticksrelmultstdev_dy_12to16,nnetticksrelmultstdev_dy_17to21,nnetticksrelmultstdev_dy_1to3,nnetticksrelmultstdev_dy_4to6,nnetticksrelmultstdev_dy_7to11,nnetticksrelmultstdev_lst15_0,nnetticksrelmultstdev_lsthrx15_0,nnetticksrelmultstdev_md_1,nnetticksrelmultstdev_nt_0,nnetticksrelmultstdev_nt_1,nnetticksrelmultstdev_nt_12to16,nnetticksrelmultstdev_nt_17to21,nnetticksrelmultstdev_nt_1to3,nnetticksrelmultstdev_nt_4to6,nnetticksrelmultstdev_nt_7to11,nnetticksrelmultstdev_pm_1,nnetticksrelmultstdev_toxhr_0,nnetticksrelrrsign_am_1,nnetticksrelrrsign_dy_12to16,nnetticksrelrrsign_dy_17to21,nnetticksrelrrsign_dy_1to3,nnetticksrelrrsign_dy_4to6,nnetticksrelrrsign_dy_7to11,nnetticksrelrrsign_lst15_0,nnetticksrelrrsign_lsthrx15_0,nnetticksrelrrsign_md_1,nnetticksrelrrsign_nt_0,nnetticksrelrrsign_nt_1,nnetticksrelrrsign_nt_12to16,nnetticksrelrrsign_nt_17to21,nnetticksrelrrsign_nt_1to3,nnetticksrelrrsign_nt_4to6,nnetticksrelrrsign_nt_7to11,nnetticksrelrrsign_pm_1,nnetticksrelrrsign_toxhr_0,nsameticksrelmultstdev_am_1,nsameticksrelmultstdev_dy_12to16,nsameticksrelmultstdev_dy_17to21,nsameticksrelmultstdev_dy_1to3,nsameticksrelmultstdev_dy_4to6,nsameticksrelmultstdev_dy_7to11,nsameticksrelmultstdev_lst15_0,nsameticksrelmultstdev_lsthrx15_0,nsameticksrelmultstdev_md_1,nsameticksrelmultstdev_nt_0,nsameticksrelmultstdev_nt_1,nsameticksrelmultstdev_nt_12to16,nsameticksrelmultstdev_nt_17to21,nsameticksrelmultstdev_nt_1to3,nsameticksrelmultstdev_nt_4to6,nsameticksrelmultstdev_nt_7to11,nsameticksrelmultstdev_pm_1,nsameticksrelmultstdev_toxhr_0,nsameticksrelrrsign_am_1,nsameticksrelrrsign_dy_12to16,nsameticksrelrrsign_dy_17to21,nsameticksrelrrsign_dy_1to3,nsameticksrelrrsign_dy_4to6,nsameticksrelrrsign_dy_7to11,nsameticksrelrrsign_lst15_0,nsameticksrelrrsign_lsthrx15_0,nsameticksrelrrsign_md_1,nsameticksrelrrsign_nt_0,nsameticksrelrrsign_nt_1,nsameticksrelrrsign_nt_12to16,nsameticksrelrrsign_nt_17to21,nsameticksrelrrsign_nt_1to3,nsameticksrelrrsign_nt_4to6,nsameticksrelrrsign_nt_7to11,nsameticksrelrrsign_pm_1,nsameticksrelrrsign_toxhr_0,qe_fwd1_open_0,qe_nt0_open_0,qe_prv1_open_0,qe_prv2_open_0,rel2retcfrtxm_am_1,rel2retcfrtxm_dy_12to16,rel2retcfrtxm_dy_17to21,rel2retcfrtxm_dy_1to3,rel2retcfrtxm_dy_4to6,rel2retcfrtxm_dy_7to11,rel2retcfrtxm_lst15_0,rel2retcfrtxm_lsthrx15_0,rel2retcfrtxm_md_1,rel2retcfrtxm_nt_0,rel2retcfrtxm_nt_1,rel2retcfrtxm_nt_12to16,rel2retcfrtxm_nt_17to21,rel2retcfrtxm_nt_1to3,rel2retcfrtxm_nt_4to6,rel2retcfrtxm_nt_7to11,rel2retcfrtxm_pm_1,rel2retcfrtxm_toxhr_0,relavgts70tr_close_1,relavgts70tr_close_12to16,relavgts70tr_close_17to21,relavgts70tr_close_1to3,relavgts70tr_close_4to6,relavgts70tr_close_7to11,relavgts70tr_now_0,relntrds70tr_am_1,relntrds70tr_dy_12to16,relntrds70tr_dy_17to21,relntrds70tr_dy_1to3,relntrds70tr_dy_4to6,relntrds70tr_dy_7to11,relntrds70tr_lst15_0,relntrds70tr_lsthrx15_0,relntrds70tr_md_1,relntrds70tr_nt_0,relntrds70tr_nt_1,relntrds70tr_nt_12to16,relntrds70tr_nt_17to21,relntrds70tr_nt_1to3,relntrds70tr_nt_4to6,relntrds70tr_nt_7to11,relntrds70tr_pm_1,relntrds70tr_toxhr_0,relvol_am_1,relvol_dy_12to16,relvol_dy_17to21,relvol_dy_1to3,relvol_dy_4to6,relvol_dy_7to11,relvol_lst15_0,relvol_lsthrx15_0,relvol_md_1,relvol_nt_0,relvol_nt_1,relvol_nt_12to16,relvol_nt_17to21,relvol_nt_1to3,relvol_nt_4to6,relvol_nt_7to11,relvol_pm_1,relvol_toxhr_0,rrirpnxm_am_1,rrirpnxm_dy_12to16,rrirpnxm_dy_17to21,rrirpnxm_dy_1to3,rrirpnxm_dy_4to6,rrirpnxm_dy_7to11,rrirpnxm_lst15_0,rrirpnxm_lsthrx15_0,rrirpnxm_md_1,rrirpnxm_nt_0,rrirpnxm_nt_1,rrirpnxm_nt_12to16,rrirpnxm_nt_17to21,rrirpnxm_nt_1to3,rrirpnxm_nt_4to6,rrirpnxm_nt_7to11,rrirpnxm_pm_1,rrirpnxm_toxhr_0,rrretstr_am_1,rrretstr_dy_12to16,rrretstr_dy_17to21,rrretstr_dy_1to3,rrretstr_dy_4to6,rrretstr_dy_7to11,rrretstr_lst15_0,rrretstr_lsthrx15_0,rrretstr_md_1,rrretstr_nt_0,rrretstr_nt_1,rrretstr_nt_12to16,rrretstr_nt_17to21,rrretstr_nt_1to3,rrretstr_nt_4to6,rrretstr_nt_7to11,rrretstr_pm_1,rrretstr_toxhr_0,rvdelta_am_1,rvdelta_dy_12to16,rvdelta_dy_17to21,rvdelta_dy_1to3,rvdelta_dy_4to6,rvdelta_dy_7to11,rvdelta_lst15_0,rvdelta_lsthrx15_0,rvdelta_md_1,rvdelta_nt_0,rvdelta_nt_1,rvdelta_nt_12to16,rvdelta_nt_17to21,rvdelta_nt_1to3,rvdelta_nt_4to6,rvdelta_nt_7to11,rvdelta_pm_1,rvdelta_toxhr_0,totrisklog_open_0,tr_am_1,tr_dy_12to16,tr_dy_17to21,tr_dy_1to3,tr_dy_4to6,tr_dy_7to11,tr_lst15_0,tr_lsthrx15_0,tr_md_1,tr_nt_0,tr_nt_1,tr_nt_12to16,tr_nt_17to21,tr_nt_1to3,tr_nt_4to6,tr_nt_7to11,tr_pm_1,tr_toxhr_0".split(",")
# train_df = helper.get_df(train_start, train_end, x_cols, FILE_PATH)
# print(train_df.head())


In [12]:
list_of_interacting_terms = [["relvol_nt_0","rrirpnxm_nt_0" ], 
                             ["relvol_lst15_0","rrirpnxm_lst15_0" ],
                             ["relvol_lsthrx15_0","rrirpnxm_lsthrx15_0" ],
                             ["relvol_toxhr_0","rrirpnxm_toxhr_0" ] ]
train_df2 = helper.get_df_with_interaction_terms(train_df,list_of_interacting_terms)
train_df2 = train_df2.drop(["relvol_nt_0", "relvol_lst15_0","relvol_lsthrx15_0", "relvol_toxhr_0"], axis=consts.COL)
train_df2.to_csv(CLEANED_DATA_PATH, index_label=False)
train_df2.head()

,cftorrrelstd_open_0,liqlog_open_0,llirpnxm_am_1,llirpnxm_dy_12to16,llirpnxm_dy_17to21,llirpnxm_dy_1to3,llirpnxm_dy_4to6,llirpnxm_dy_7to11,llirpnxm_lst15_0,llirpnxm_lsthrx15_0,...,tr_nt_1to3,tr_nt_4to6,tr_nt_7to11,tr_pm_1,tr_toxhr_0,tonight,"('relvol_nt_0', 'rrirpnxm_nt_0')","('relvol_lst15_0', 'rrirpnxm_lst15_0')","('relvol_lsthrx15_0', 'rrirpnxm_lsthrx15_0')","('relvol_toxhr_0', 'rrirpnxm_toxhr_0')"
0,-0.039664,0.937606,0.001552,-0.002033,-0.010404,0.004161,-0.004310,0.010731,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,-0.000073,0.000000,0.000000,0.000000,0.000000
1,-0.157931,1.625474,-0.000785,-0.006762,-0.013537,-0.004079,0.000855,0.000974,0.000721,0.001569,...,0.0,0.0,0.000000,0.0,0.0,-0.004306,-0.003343,0.002020,0.001633,-0.032020
2,-0.669988,-1.475189,0.000309,-0.011166,-0.007546,0.004229,-0.004087,0.003461,-0.000018,-0.001450,...,0.0,0.0,0.000000,0.0,0.0,0.004177,-0.000000,0.000809,-0.003780,0.007307
3,1.558111,-1.694274,0.000500,0.039606,0.003121,-0.011830,0.002213,-0.005037,0.000000,0.000000,...,0.0,0.0,0.007313,0.0,0.0,-0.000073,0.000000,0.000000,0.000000,0.000000
4,-0.121591,-1.274620,-0.001676,0.007767,0.013411,0.007256,0.001081,-0.003754,0.000264,-0.000502,...,0.0,0.0,0.000000,0.0,0.0,-0.007763,0.001246,-0.000909,0.000325,-0.001265
